## Carga de datos, almacenamiento y formatos de archivo

In [1]:
import pandas as pd
import numpy as np

### Lectura y escritura de datos en formato de texto
Pandas presenta una serie de funciones para leer datos tabulares como un objeto
DataFrame. `pandas.read_csv` es uno de los más utilizados.

**Indexación**: puede tratar una o más columnas como el DataFrame devuelto, y si desea
obtener nombres de columna del archivo, argumentos que proporcione o no.

**Inferencia de tipo y conversión de datos**: incluye las conversiones de valor
definidas por el usuario y lista personalizada de marcadores de valores faltantes.

**Análisis de fecha y hora**: incluye una capacidad de combinación, que incluye la
combinación de información de fecha y hora distribuida en varias columnas en una sola
columna en el resultado.

**Iterando**: soporte para iterar sobre trozos de archivos muy grandes.

**Problemas de datos impuros**: incluye omitir filas o pie de página, comentarios u
otras cosas menores como datos numéricos con miles separados por coma.

In [2]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv("ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# un archivo no siempre tendrá una fila de encabezados
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
# dejar que pandas asigne nombre a las columnas
pd.read_csv("ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# asignar nombre a las columnas
pd.read_csv("ex2.csv", names=list("abcd"))

,a,b,c,d
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [7]:
# indicar que la columna d sea índice
names = ["a", "b", "c", "d", "message"]
pd.read_csv("ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [8]:
"""
formar un índice jerárquico: pasar una lista de números
o nombres
"""
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed = pd.read_csv("csv_mindex.csv",
                    index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Un archivo no simpre puede tener un delimitador fijo:

In [10]:
# el archivo tiene una cantidad variable de espacios en blanco
!cat ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [11]:
# usar una expresión regular junto a pd.read_csv
result = pd.read_csv("ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Omitir la primera, tercera y cuarta fila de un archivo con `skiprow`:

In [12]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [13]:
pd.read_csv("ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


El manejo de valores faltantes es una parte importante y frecuentemente matizado del
proceso de lectura de archivos. Los datos faltantes generalmente no están presentes
(cadenas vacías) o marcados por algunos centinelas (placeholder). Por defecto, pandas
usa un conjunto de centinelas comunes, como `NA` y `NULL`:

In [14]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [15]:
result = pd.read_csv("ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [17]:
result = pd.read_csv("ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


`keep_default_na` deshabilita la representación predeterminada para los `NA`:

In [18]:
result2 = pd.read_csv("ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [19]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [20]:
result3 = pd.read_csv("ex5.csv", keep_default_na=False,
                     na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [21]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [22]:
# especificar diferentes centinelas NA para cada columna en un diccionario
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("ex5.csv", na_values=sentinels,
           keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Lectura de archivos de texto en piezas

Antes de mirar un archivo grande, hacemos que la configuración de visualización de
pandas sea más compacta:

In [23]:
pd.options.display.max_rows = 10

In [24]:
result = pd.read_csv("ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


Las marcas de elipsis `...` indican que se han omitido filas en medio del DataFrame.

In [25]:
# leer un pequeño número de filas con nrows
pd.read_csv("ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [26]:
# leer un archivo en pedazos con chunksize
chunker = pd.read_csv("ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

`TextFileReader` devuelto por `pd.read_csv` le permite iterar sobre las partes del
archivo de acuerdo con `chunksize`. Por ejemplo, podemos iterar sobre `ex5.csv`,
agregando los recuentos de valor en la columna `"key"`:

In [27]:
chunker = pd.read_csv("ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

In [28]:
# resultado
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Escribir datos de formato de texto
Los datos también se pueden exportar a un formato delimitado.

In [29]:
data = pd.read_csv("ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Usando el método `to_csv`, podemos escribir los datos en un archivo separado por comas:

In [30]:
data.to_csv("out.csv")
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [31]:
import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Los valores faltantes aparecen como cadenas vacías en la salida. Es posibe que desee
denotarlos por algún otro valor centinela:

In [32]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [33]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Sin otras opciones especificadas, se escriben las etiquetas de filas y columna.
Ambos pueden ser deshabilitados:

In [34]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


### Trabajando con otros formatos delimitados
Es posible cargar la mayoría de las formas de datos tabulares desde el disco utilizando
funciones como `pd.read_csv`. En algunos casos, sin embargo, puede ser necesario un
procesamiento manueal. No es raro recibir un archivo con una o más líneas malformadas.

In [35]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


Para cualquier archivo con un delimitadr de un solo carácter, puede usar el módulo `csv`
incorporado de Python.

In [36]:
import csv

f = open("ex7.csv")

reader = csv.reader(f)

Al esterilizar a través del lector como un archivo produce listas de valores con
cualquier carácter de comillas eliminado:

In [37]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [38]:
f.close()

A partir de ahí, depende de usted hacer las disputas necesarias para poner los datos en
la forma que necesita. Ejemplo:

In [39]:
with open("ex7.csv") as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [40]:
# dividir las líneas para encabezado y las líneas de datos
header, values = lines[0], lines[1:]

In [41]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Los archivos CSV vienen en muchos sabores diferentes. Para definir un nuevo formato con
un delimitador diferente, una convención de cotización de cadena o terminador de línea,
podríamos definir una subclase simple de `csv.Dialect`:

In [42]:
class MyDialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [43]:
f = open("ex7.csv")
reader = csv.reader(f, dialect=MyDialect)

También pasar parámetros de dialectos individuales a CSV como palabras clave para
`csv.reader` sin tener que definir una subclase:

In [44]:
reader = csv.reader(f, delimiter="|")

Para escribir archivos delimitados manualmente se puede usar `csv.writer`.

In [45]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=MyDialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

In [46]:
!cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON Data
JSON (JavsScript Object Notation) se ha convertido en uno de los formatos estándar
para enviar datos mediante solicitud HTTP entre navegadores web y otras aplicaciones.
Es un formato de datos mucho más libre que un formulario de texto tabular como CSV.
Ejemplo:

In [47]:
obj = """
{"name": "Wes",
"cities_lives": ["Akron", "Nashvile", "New York", "San Francisco"],
"pet": null,
"siblings": [{"name": "Scoot", "age": 34, "hobbies": ["guitars", "soccer"]},
{"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [48]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lives': ['Akron', 'Nashvile', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scoot', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [49]:
# json.dumps convierte un objeto Python a JSON
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lives": ["Akron", "Nashvile", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scoot", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

La forma en que convierta un objeto JSO o una lista de objetos en un DataFrame o alguna
otra estructura de datos para su análisis dependerá de usted. Convenientemente, puede
pasar una lista de diccionario (que anteriormente era un objeto JSOn) al constructor
DataFrame y seleccionar un subconjunto de los campos de datos:

In [50]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scoot,34
1,Katie,42


`pd.read_json` puede convertir automáticamente conjuntos de datos JSON en arreglos
específicos en una Serie o DataFrame. Por ejemplo:

In [51]:
!cat example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


Las opciones predeterminadas para `pd.read_json` suponen que cada objeto en la matriz
JSON es una fila en la tabla:

In [52]:
data = pd.read_json("example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


Si necesita exportar datos de pandas a JSON, una forma es usar el método `to_json`
en Series y DataFrame:

In [53]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [54]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

### XML y HTML: Web Scraping
Pandas Tiene una función incorporada, `pd.read_html`, que utiliza todas estas
bibliotecas para analizar automáticamente tablas de archivos HTML como objetos
DataFrame.

In [55]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable


La función `pd.read_html` tiene varias opciones, pero por defecto busca e intenra
analizar todos los datos tabulares contenidos en etiquetas `<table>`. El resultado
es una lista de objetos DataFrame:

In [56]:
tables = pd.read_html("fdic_failed_bank_list.html")
len(tables)

1

In [57]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


A partir de aquí, por ejemplo, podríamos proceder a hacer algunos análisis y limpieza
de datos, como calcular el número de quiebras bancarias por año:

In [58]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

### Análisis XMl con `lxml.objectify`
XML es otro formato de datos estructurado común que admite datos jerárquicos anidados
como metadatos.

XLM y HTML son estructuralmente similares, pero XML es más general.

Durante muchos años, la Autoridad Metropolitana de Transporte deNueva York (MTA)
publicó una serie de datos sobre sus servivios de autobuses y trenes en formato XML.
Aquí veremos los datos de rendimiento, que están contenidos en un conjunto de archivos
XML. Cada servicio de tres o autobús tienen un archivo diferente (como _Rendimiento_MNR.xml
para el Metro-North Railroad) que contiene datos mensuales como una serie de registros
XML que se ven así:

Usando `lxml.objectify` analizamos el archivo y obtenemos una referencia al nodo raíz
del archivo XML con `getroot`:

In [59]:
from lxml import objectify

path = "Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)
    
root = parsed.getroot()

`root.INDICATOR` devuelve un generador que produce cada un elemento `<INDICATOR>`
XML. Para cada registro, podemos rellenar un diccionario de nombres de etiquetas
(como `YTD_ACTUAL`) a valores de datos (excluyendo algunas etiquetas):

In [60]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
              "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)


Por último, convertir esta lista de diccionarios en un DataFrame:

In [61]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


La función `pd.read_xml` convierte este proceso en una expresión de una línea:

In [62]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


### Formatos de datos binarios
Una forma sencilla de almacenar (o serializar) los datos en formato binario es
usar el módulo incorporado `pickle`. Todos los objetos de pandas tienen un método
`to_pickle` que escribe los datos en el disco en formato pickle:

In [63]:
frame = pd.read_csv("ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [64]:
frame.to_pickle("frame_pickle")

Los archivos pickle son generalmente legibles solo en Python. Puede leer cualquier
objeto
almacenado en un archivo utilizando `pickle` directamente , o incluso más
convenientemente usando `pd.read_pickle`:

In [65]:
pd.read_pickle("frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Pandas tiene soporte incorporado para varios otros formatos de datos binarios de código
abierto, como HDF5, ORC y Apache Parquet. Por ejemplo, `pip install {pyarrow, parquet, 
fastparquet}` 
puede leer archivos parquet con `pd.read_parquet`:

In [66]:
#!pip install pyarrow, parquet, fastparquet

In [67]:
import fastparquet

fec = pd.read_parquet("fec.parquet")

### Lectura de archivos Microsoft Excel

Pandas también admite la lectura de datos tabulares almacenados en archivos Excel
2003 (y superiores) utilizando `pd.ExcelFile` o `pd.read_excel`. Internamente, estas
herramientas utilizan los paquetes adicionales `xlrd` y `openpyxl` para leer archivos
XLS de estilo antiguo y XLSX más nuevos.

In [68]:
!pip install openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


Pasar usar `pd.ExcelFile`, crear una instancia pasando una archivo como ruta:

In [69]:
xlsx = pd.ExcelFile("ex1.xlsx")

Este objeto puede mostrarle la lista de nombres de hojas disponibles en el archivo:

In [70]:
xlsx.sheet_names

['Sheet1']

Los datos almacenados en una hoja se puede leer en DataFrame con `parse`:

In [71]:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


Esta tabla de Excel tiene una columna de índice, por lo que podemos indicar con el
argumento `index_col`:

In [72]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Si está leyendo varias horas en un archivo, entonces es más rapido crear `pd.ExcelFile`,
pero también puede simplemente pasar el nombre del archivo a `pd.read_excel`:

In [73]:
frame = pd.read_excel("ex1.xlsx", sheet_name="Sheet1", index_col=0)
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Para escribir datos en pandas en formato Excel, primero debe crear un `ExcelWriter`,
luego escribir datos usando el método de pandas `to_excel`:

In [74]:
writer = pd.ExcelWriter("ex2.xlsx")
frame.to_excel(writer, "Shit01")
writer.close()

In [75]:
pd.ExcelFile("ex2.xlsx").sheet_names

['Shit01']

In [76]:
pd.read_excel("ex2.xlsx")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


También puede pasar una ruta de archivo a `to_excel` y evitar `ExcelWriter`:

In [77]:
frame.to_excel("ex2_2.xlsx")

In [78]:
pd.read_excel("ex2_2.xlsx")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


### Usar el formato HDF5
HDF5 es un formato de archivo respetado destinado a almacenar grandes cantidades de
datos de matriz científica. En comparación con los formatos más simples, HDF5 admite
la compresión sobre la marcha con una variedad de modos de compresión, lo que permite
que los datos con patrones repetidos se almacenen de manera más simple. HDF5 puede ser
una buena opción para trabajar con conjunto de datos que no encajan en la memoria, ya
que pueden leer y escribir eficientemente pequeñas secciones de matrices muchos más
grandes.

In [79]:
#!pip install tables

Pandas proporciona una interfaz de alto nivel que simplifica el almacenamiento de
Series y DataFrame. La función `HDF5Store` funciona como un diccionario y maneja los
detalles de bajo nivel:

Los objetos contenidos en el archivo HDF5 se pueden recuperar con la misma API similar a
un diccionario:

`HDF5Store` admite dos esquemas de almacenamiento, `"fidex"` y `"table"` (el valor
predeterminado es `"fixed"`). Este último es más lento, pero admite operaciones de
consulta utilizando una sintaxis especial:

### Interactuar con APIs web
Muchos sitos web tienen API públicas que proporcionan fuentes de datos a través de
JSON o algún otro formato.

In [80]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


Encontrar los últimos 30 problemas de Pandas en Github, podemos hacer una solicitud
`GET` utilizando el módulo `requests`:

In [81]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"

resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

Es buena práctica llamar siempre a `raise_for_status` después de usar `requests.get`
para verificar errores HTTP.

In [82]:
data = resp.json()
type(data)

list

In [83]:
data[0]["title"]

'ENH: Allow passing `read_only`, `data_only` and `keep_links` arguments to openpyxl using `engine_kwargs`'

Cada elemento en `data` es un diccionario que contiene todos los datos encontrados en
una página de emisión de GitHub (exceto los comentarios). Podemos pasar `data`
directamente a `pd.DataFrame` y extrar campos de interés:

In [84]:
issues = pd.DataFrame(data, columns=["number", "title",
                                    "labels", "state"])
issues

,number,title,labels,state
0,55807,"ENH: Allow passing `read_only`, `data_only` an...",[],open
1,55806,TST: parametrize over unit,[],open
2,55805,ENH/BUG: infer reso in array_strptime,[],open
3,55803,DOC: na_values defaults for read_csv don't mat...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
4,55802,BUG: Pyarrow Float dataframe -> to_numpy yield...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,55773,BUG: Index name changed when using loc[] with ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,55772,BUG: `DataFrame.resample` changing index type ...,[],open
27,55767,PERF: Surprisingly slow nlargest with duplicat...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
28,55765,BUG: describe started rounding reported percen...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


### Interactuar con Base de Datos
En una configuración comercial, es posible que muchos datos no se almacenen en archivos
de texto o Excel. Las bases de datos relacionales basadas en SQL (como SQL Server,
PostreSQL y MySQL) se utilizan ampliamente, y muchas bases de datos alternativas se han
vuelto bastante populares. La elección de la base de datos generalmente depende del
rendimiento, la integridad de los datos y las necesidades de escalabilidad de una
aplicación.

In [85]:
import sqlite3

query = """
CREATE TABLE IF NOT EXISTS test(
a VARCHAR(20),
b VARCHAR(20),
c REAL,
d INTEGER);
"""

con = sqlite3.connect("mydata.sqlite")

con.execute(query)
con.commit()

In [86]:
# intertar algunas filas de datos
data = [("Atlanta", "Georigia", 1.25, 6),
       ("Tallahassee", "Florida", 2.6, 3),
       ("Sacramento", "California", 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

La mayoría de los controladores SQL de Python devuelven una lista de tuplas al
seleccionar datos de una tabla:

In [87]:
cursor = con.execute("SELECT * FROM test")

rows = cursor.fetchall()
rows

[('Atlanta', 'Georigia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georigia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georigia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georigia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Puede pasar la lista de tuplas al constructor DataFrame, pero también necesita los
nombres de columna, contenidos en los cursores `description`.

In [88]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georigia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georigia,1.25,6
4,Tallahassee,Florida,2.60,3
...,...,...,...,...
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5
9,Atlanta,Georigia,1.25,6
10,Tallahassee,Florida,2.60,3


El proyecto SQLAlchemy es un popular kit de herramientas Python SQL que abstrae muchas
de las diferencias comunes entre las bases de datos SQL. Pandas tiene una función
`read_sql` que le permite leer datos fácilmente desde una conexión general de
SQLAlchemy.

In [89]:
#!pip install sqlalchemy
!pip install --force-reinstall 'sqlalchemy < 2.0.0'

Defaulting to user installation because normal site-packages is not writeable
  Using cached SQLAlchemy-1.4.50-cp39-cp39-linux_i686.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.50
    Uninstalling SQLAlchemy-1.4.50:
      Successfully uninstalled SQLAlchemy-1.4.50


In [90]:
# usando sqlalchemy
import sqlalchemy as sqla

db = sqla.create_engine("sqlite:///mydata.sqlite")

pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georigia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georigia,1.25,6
4,Tallahassee,Florida,2.60,3
...,...,...,...,...
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5
9,Atlanta,Georigia,1.25,6
10,Tallahassee,Florida,2.60,3
